#### Problem Statement 
 Stayze is an online market company that provides lodging and primary homestays.The company doesnot  own any real estate or property but it recieves some brokerage for each booking. The activities of hosts ( they rent out their property that shed light on the area, availiability, price , and other amenities) and travellers (display their reviews ) resulted in a rich database. 
The stakeholders are : 
1. The CEO of the company since it receives  commission from each booking.
2. The hosts whose properties are listed
3. Finance department (for ideal investment) and marketing department(for marketing initiatives).

Th stakeholders want to utilise this data efficiently to know the ideal prices at which the properties could be rented. Further,another  objective is  to understand the behavior of customers and providers that would guide in formulating the marketing initiatives and implementing the innovative additional services.


#### Business Objective

To predict the prices at which the listed properties could be rented since it would help the stakeholders to decide about the ideal investment . 

#### Data Science Objective 
Since the problem involves predicting the rental price, it reflects the case of supervised machine learning . We could consider the machine leraning alogrithms such as Linear Regression and Random Forest with cross validation to predict the prices based on the given variables.

#### Hypothesis Building : TARGET VARIABLE : Price
 Based on the given dataset , we could hypothesis the following :
a) Does the variables such as location , area ,availability of property and nearby amenities impact the rental price?
b) Does the minimum nights clause decrease the rental price ?
c) Does the seasonality impact the prices?
d) Is there any correlation between rental demand and prices?

In [3]:
from google.colab import drive
drive.mount('/content/drive/My Drive/Team Rocket/Train.csv')

ModuleNotFoundError: No module named 'google'

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import warnings
warnings.filterwarnings('ignore')

## TRAINING DATA SET

### Step 1 : Reading and Inspecting the data set 

In [ ]:
# Importing the training file
df = pd.read_csv('/content/drive/My Drive/Team Rocket/Train.csv')
# Looking at the first five rows
df.head()

In [ ]:
df.info(verbose=True)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

We can infer following informationfrom the above describe function:
1. Price :The maximum price is 10000  where as the mean price is 153. It clearly indicates the presence of outliers. 
2. Similar is the case with the variables such as minimum_nights and number of reviews

### Step 2 : Data Quality Checks and Handling Missing Values

In [ ]:
# columnwise null count for application data
df.isnull().sum()

In [ ]:
#column-wise null percentage of the rental data
df_col = round((df.isnull().sum()/len(df.index))*100,2)
print("column wise percentage in rental data", df_col)

 <b> We can notice that the variables last_review and reviews_per_month  have some NaN values.These are numeric varibles and this we can impute these NAN values with the appropriate measures such as mean and median. Further host_name and name alos have  few null values that could be  filled with the most frequent value since these are categorical variable. </b>

### 1. last_review

In [ ]:
df.last_review.value_counts()

In last_review column there are 6982 null value rows.This accounts to about 20% of the missing values.The missing value of this variable can be handled by imputing with the  most frequent valie i.e 2019-06-23

In [ ]:
df['last_review'].fillna('2019-06-23',inplace = True)

In [ ]:
# Lets again look at the column-wise null percentage of the rental data
df_col = round((df.isnull().sum()/len(df.index))*100,2)
print("column wise percentage in rental data", df_col)

### 2. reviews_per_month

In [ ]:
df.reviews_per_month.describe()

Here we can treat the missing values by imputing it with '0' since if we look at the column 'number of reviews' there are certain rows with 0 value. Since the reviews_per_month is calculated based on number of reviwes column , we can handled the missing values by replacing it with 0

In [ ]:
df.reviews_per_month = df.reviews_per_month.fillna(0)

In [ ]:
# Lets again look at the column-wise null percentage of the rental data
df_col = round((df.isnull().sum()/len(df.index))*100,2)
print("column wise percentage in rental data", df_col)

### 3. host_name


In [ ]:
df.host_name.value_counts()

The missing values in the column hist_name are almost negligible i.e 0.04%. Howeevr, still we can impute them with the frequent name occuring in the variable i.e Michael. Lets impute this value. 

In [ ]:
df.host_name = df.host_name.fillna("Michael")

In [ ]:
# Lets again look at the column-wise null percentage of the rental data
df_col = round((df.isnull().sum()/len(df.index))*100,2)
print("column wise percentage in rental data", df_col)

### 4. name

In [ ]:
df.name.value_counts()

Like host_name , here again  we can impute them with the frequent name occuring in the variable i.e Michael. Lets impute this value. 

In [ ]:
df.name = df.host_name.fillna("Home away from home")

In [ ]:
# Lets again look at the column-wise null percentage of the rental data
df_col = round((df.isnull().sum()/len(df.index))*100,2)
print("column wise percentage in rental data", df_col)

#### Creating a column with lat,long tuples as we need this format to be the input for our haversine func.
column lat_long_original added in train dataset and column lat_long added in airport dataset

In [ ]:
airport = pd.read_csv('airports.csv')
airport.head()

In [ ]:
df_airport = pd.read_csv('train.csv')
df_airport.head()

In [ ]:
df_airport['lat_long_original'] = list(zip(df_airport.latitude,df_airport.longitude))

In [ ]:
airport['lat_log']= list(zip(airport.latitude_deg, airport.longitude_deg))

In [ ]:
airport['lat_log']

In [ ]:
df_airport['lat_long_original']

#### Adding a column for feature(airport_facility) and distance from the airport if within given range of 3.5 miles.

In [ ]:
df_airport['airport_facility']=pd.Series(data=0)
df_airport['airport_distance']=pd.Series(data=0)

In [ ]:
array_airport = []
for i in airport['lat_log']:
    array_airport = array_airport + [i]
print(array_airport) 

In [ ]:
array_data = []
for i in df_airport['lat_long_original']:
    array_data = [i] *2
    listofmiles =  np.round_(haversine_vector(array_data,array_airport,unit='mi'),decimals = 3) 
    listofmiles = list(filter(lambda x: x <3.5 and x>0 ,listofmiles))
    if len(listofmiles)!=0:
        df_airport.loc[df_airport['lat_long_original']==i, 'airport_facility'] = True
        df_airport.loc[df_airport['lat_long_original']==i, 'airport_distance'] = min(listofmiles)
    else:
        df_airport.loc[df['lat_long_original']==i, 'airport_facility'] = False 

In [ ]:
plt_2 = df_airport['airport_facility'].value_counts()

In [ ]:
airports_2 = sns.barplot(plt_2.index, plt_2.values)

In [ ]:
airport = df_airport[df_airport['airport_facility']==True]
print(airport.shape)
airport_grp = airport['neighbourhood_group'].value_counts()
airport_grp
fig1 = sns.barplot(x = airport_grp.index, y = airport_grp)
fig1.set(xlabel='neighbourhood_group', ylabel='count', title='Listings within 3.5 miles of airport')

### Step 3  EDA : UNIVARIATE ANALYSIS

In [ ]:
#for categorical variables
categories = df.select_dtypes(include = 'object')
list(enumerate(categories))


In [ ]:
plt.figure(figsize=(10,7))
sns.countplot('neighbourhood_group',data=df)
plt.show()

The above graph indicates that the areas staten Island and Bronx are under-represented in the data set. On the other hand Manhattan and Brooklyn are in high demand which are favorable palaces for house renting. 

In [ ]:
df.neighbourhood.value_counts().head(10).plot.bar()

The neghborhood graph indicates that williamsburg and Beddford-Stuyvesant are the top two places for high rental demand. 

#### Thus the from the neighbourhood_group and neighbourhoof, we can infer the following results:
Following combination are favorable place for house renting : 
1. Brooklyn with neighborhood like Willamburg , Bedford-Stuyvesant ,Bushwick.
2. Manhattan with neighborhood like Harlem, Upper West Side and Hell's Kitchen.


In [ ]:
df.host_name.value_counts().head(10).plot.bar()

In [ ]:
plt.figure(figsize=(7,7))
sns.countplot('room_type',data=df)
plt.show()

The above figure illustrates that rental demand for shared room is less as compared to the provate room and entire apartment. 


In [ ]:
#Continuous = ['latitude','longitude','price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
Continuous = df.select_dtypes(include = 'number')
list(enumerate(Continuous))


In [ ]:
list(enumerate(Continuous))
plt.figure(figsize=(15,20))  
for i in enumerate (Continuous):
    plt.subplot(6,2,i[0]+1)
    plt.tight_layout(pad=4.0)
    sns.distplot(df[i[1]], hist = True , color = 'r')
    plt.xticks(rotation=90)
plt.show()

In [ ]:
def detect_skew(df):
    sk = df.skew()
    skewed = sk[sk>0.6]
    print('Greater than 0.6 \n',skewed)
detect_skew(Continuous)

Since most of the numerical variables are skewed rightwards , we decided to normalize these features by using logp1.

#### Correlation Analysis

In [ ]:
# Let's see the correlation matrix 
plt.figure(figsize = (20,10))       
sns.heatmap(df.corr(),annot = True,cmap = 'coolwarm', linecolor = 'white', linewidths=0.1)
plt.show()

The heat map delineates that correlation is low-medium (0.58) between number_of_reviews and reviews_per_month. In case of all other variables, the correlation is less than 0.50. Accordingly, there is no need to drop the columns.

### Bivariate Analysis

In [ ]:
plt.figure(figsize = (15,5))
sns.jointplot(x=df.latitude,y=df.longitude,size=10)
plt.ylabel ('Longitude',fontsize=12)
plt.xlabel('Latitude',fontsize =12)
plt.show()

The above Joinplot function helps to check the concentration of data . For latitude between 40.6 and 40.8 there are many houses , which would mean that  it’s an ideal location for rental .On the other hande, for  longitude we can see that concentration is high between -73.9 to -74 which indicates that most of the rental demand  has been for this particular location.


In [ ]:
plt.scatter(df.price,df.longitude)
plt.title ('Price vs Location of the area')

The above scatter plot tells us about the location of  the rental houses in terms of longitude and it gives us quite an interesting observation that  location between  -73.9 to -74 rent houses at much higher prices.

In [ ]:
plt.scatter(df.price,df.latitude)
plt.title ('Price vs Location of the area')

The above scatter plot tells us about the location of the rental houses in terms of latitude and we can infer that the location between 40.6 to 40.8 rent houses at much higher prices.

In [ ]:
plt.scatter(df.price,df.minimum_nights)
plt.title ('Price vs minimum_nights')

#### THE ABOVE ANALYSIS OF LONGITUDE AND LATITUDE clearly indicates that since the most of the rental demand is concentrated between  -73.9 to -74  (longitude ) and 40.6 and 40.8 (latitude), the price is also high in this location. Accordingly, more investment could be considered in this location.

In [ ]:
plt.figure(figsize = (10,5))
sns.scatterplot(x=df.latitude,y=df.longitude, hue = df.neighbourhood_group)
plt.ylabel ('Longitude',fontsize=12)
plt.xlabel('Latitude',fontsize =12)
plt.show()

The above figure indicates that in addition to Manhattan and Brooklyn, the rental demand is high in their adjacent areas such as Queens and Bronx.


In [ ]:
plt.tight_layout(pad=4.0)
sns.countplot(df.neighbourhood_group, hue =  df.room_type)
plt.xticks(rotation=90)
plt.show()

We can infer following from the above graph:
1. While the Entire home/Apartment and Private room are more in demand  in Manhattan, the demand of private room and entire home apartment are almost same in Brooklyn.
2. Shared Room are not much in demand in any of the neighborhood.


In [ ]:
plt.figure(figsize = (10,5))
plt.tight_layout(pad=4.0)
sns.boxplot( x = df.neighbourhood_group, y = df.price , hue = df.room_type)
plt.yscale('log')
plt.xticks(rotation=90)
plt.show()

Prices of all room_type are higher in Manhattan. This reflects more investment in this area for more revenue generation. Further, there is no need to invest in shared room type since there is no demand of such rooms.

### Identifying the Outliers

In [ ]:
# Outliers
#plt.figure(figsize = (30,10))
#for i in enumerate(Continuous):
  #  plt.subplot(3,3, i[0]+1)
   # sns.boxplot(df[i[1]])

The box plot analysis indicates the presence of outliers in almost all factors.

In [ ]:
#checking the outliers at 25%, 50%, 75%, 90%,95% and 99%
numeric = df[['latitude','longitude', 'price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count']]
numeric.describe(percentiles=[.25,.5,.75,.90,.95,.98,.99])

In [ ]:
# capping the outeliers for price
q = df['price'].quantile(0.90)
df['price'][df['price']>=q] = q
sns.boxplot(df['price'])

In [ ]:
# capping the outeliers for minimum_nights
q = df['minimum_nights'].quantile(0.80)
df['minimum_nights'][df['minimum_nights']>=q] = q
sns.boxplot(df['minimum_nights'])

In [ ]:
# capping the outeliers for price
q = df['number_of_reviews'].quantile(0.85)
df['number_of_reviews'][df['number_of_reviews']>=q] = q
sns.boxplot(df['number_of_reviews'])

In [ ]:
# capping the outeliers for 'reviews_per_month'
q = df['reviews_per_month'].quantile(0.90)
df['reviews_per_month'][df['reviews_per_month']>=q] = q
sns.boxplot(df['reviews_per_month'])

In [ ]:
# capping the outeliers for calculated_host_listings_count
q = df['calculated_host_listings_count'].quantile(0.85)
df['calculated_host_listings_count'][df['calculated_host_listings_count']>=q] = q
sns.boxplot(df['calculated_host_listings_count'])

In [ ]:
# capping the outeliers for latitude (upper range)
q = df['latitude'].quantile(0.90)
df['latitude'][df['latitude']>=q] = q
sns.boxplot(df['latitude'])


In [ ]:
# capping the outeliers for latitude(lower range)
q = df['latitude'].quantile(0.20)
df['latitude'][df['latitude']<=q] = q
sns.boxplot(df['latitude'])

In [ ]:
# capping the outeliers for longitude (upper range)
q = df['longitude'].quantile(0.90)
df['longitude'][df['longitude']>=q] = q
sns.boxplot(df['longitude'])


In [ ]:
# capping the outeliers for longitude (lower range)
q = df['longitude'].quantile(0.20)
df['longitude'][df['longitude']<=q] = q
sns.boxplot(df['longitude'])


### Step 4 : Data Preparation and adding new features 

### 1.MONTHS (SESAONALITY) : It has been extraxted from last_review column to check seasonality

In [ ]:
df['last_review'] = pd.to_datetime(df.last_review,format='%Y-%m-%d')
df['Months'] = df.last_review.dt.strftime('%m')
df.Months.value_counts()

In [ ]:
#Lets map the variable Months
df['Months'] = df['Months'].astype(int)
df['Months'] = df['Months'].map({1:'jan',2:'feb',3:'mar', 4:'apr',5:'may',6:'jun', 7:'jul',8:'aug',9:'sep',10 :'oct',11:'nov',12:'dec'})
df.Months.value_counts()

####  We observe the following variables are irrelevant . Lets drop these variables 
1. name
2. Id
3. host_name
4. last_review
5. neighbourhood

In [ ]:
df.drop(['id','name','host_name','last_review','neighbourhood'],1,inplace=True)
df.head()

### 3.  Lets us create the dummy variables for categorical values

In [ ]:
month_dummy = pd.get_dummies(df['Months'],drop_first = True)
df = pd.concat([df,month_dummy], axis =1)

room_dummy = pd.get_dummies(df['room_type'],drop_first = True)
df = pd.concat([df,room_dummy], axis =1)

neighbourhood_group_dummy = pd.get_dummies(df['neighbourhood_group'],drop_first = True)
df = pd.concat([df,neighbourhood_group_dummy], axis =1)



In [ ]:
df.head()

In [ ]:
# Drop month, room_type and neighbourhood_group as we have created the dummies for it

df = df.drop(['Months','room_type','neighbourhood_group'], axis = 1)
df.head()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (25, 15))
sns.heatmap(df.corr(), annot = True, cmap="YlGnBu")
plt.show()

In [ ]:
df.columns

***Through this exploratory data analysis and visualization project, we gained several interesting insights into the rental market***
a) Does the variables such as location , room_type ,availability of property impact the rental price? 

The EDA analysis indicates that Manhattan has the most expensive rentals as compared to the other neighbourhood.Further,the rental demand for shared room is less as compared to the provate room and entire apartment.


b) Does the minimum nights clause decrease the rental price ?

There seems to be no correlation between minimum nights and price 

c) Does the seasonality impact the prices?

The demand ( it can be inferred from the umber of reviews) shows a seasonal pattern - demand increases from May to July. It cleraly indicated that most people traveled during this month to the tourist spots.


d) Is there any correlation between rental demand and prices?

The ALYSIS OF LONGITUDE AND LATITUDE clearly indicates that since the most of the rental demand is concentrated between -73.9 to -74 (longitude ) and 40.6 and 40.8 (latitude), the price is also high in this location. Accordingly, there is strong correlation between rental demand and price.

### 5. Model Building on the Training Data : LINEAR REGRESSION 

#### In order to deal with the skweness, we have to use logp1


In [ ]:
Numeric = ['price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']

df[Numeric] = np.log1p(df[Numeric])
df



1. Performing a train-test split 

In [ ]:
from sklearn.model_selection import train_test_split


np.random.seed(0)
df_train, df_test = train_test_split(df, train_size = 0.7, test_size = 0.3, random_state = 100)

df_test.shape
df.head()


2. Dividing into X and Y sets for the model building

In [ ]:
X_train = df_train.drop(['price'],1)
y_train = df_train['price']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['latitude','longitude','price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df.head()

In [ ]:
y_train.head()

3. Feature Selection 

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
ln = LinearRegression()
ln.fit(X_train, y_train)
rfe = RFE(ln,15)
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]

4. Fitting the Model

In [ ]:
import statsmodels.api as sm


X_train_lm = X_train[['latitude', 'longitude', 'minimum_nights', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'feb', 'mar',
       'oct', 'Private room', 'Shared room', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island']]
X_train_lm = sm.add_constant(X_train_lm)

# Create a first fitted model
lr = sm.OLS(y_train, X_train_lm).fit()


In [ ]:
# Print a summary of the linear regression model obtained
print(lr.summary())

#### The R-squared value obtained is 0.53. All the variables are sigicicant at the significance level of 5%. Let have a look at the VIF to check multocollinearity.

In [ ]:
# calculate the VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
x = X_train_lm.drop(['const'],axis=1)
vif['Features'] = x.columns
vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range (x.shape[1])]
vif['VIF'] = round(vif['VIF'],2)
vif=vif.sort_values(by = 'VIF',ascending = False)
vif

### The p value of 'Staten Island' is not  significant at 5% level. Lets drop this variable in the next model. 

### Updating the model : removing variable 'longitude'


In [ ]:
# Build a linear model

X_train_new = X_train_lm.drop(['longitude'],axis=1)
X_train_new= sm.add_constant(X_train_new)
lr_1=sm.OLS(y_train, X_train_new).fit()
print (lr_1.summary())

#### R square value is  53% even after removing `longitude` variable and all other variables are significant. Lets check VIF

In [ ]:
vif = pd.DataFrame()
x = X_train_new.drop(['const'],axis=1)
vif['Features'] =x.columns
vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range (x.shape[1])]
vif['VIF'] = round(vif['VIF'],2)
vif=vif.sort_values(by = 'VIF',ascending = False)
vif

***Now as you can see, the VIFs values are very high and Queens is insignificant . So lets drop variable 'Queens'  and run the model again***

In [ ]:
X_train_lm = X_train_new.drop(['Queens'],axis=1)
X_train_lm= sm.add_constant(X_train_lm)
lr_2=sm.OLS(y_train, X_train_new).fit()
print (lr_2.summary())

In [ ]:
vif = pd.DataFrame()
x = X_train_lm.drop(['const'],axis=1)
vif['Features'] =x.columns
vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range (x.shape[1])]
vif['VIF'] = round(vif['VIF'],2)
vif=vif.sort_values(by = 'VIF',ascending = False)
vif

*** Again VIF is very high, lets remove 'latitude' column and run the model***

In [ ]:
X_train_lr = X_train_lm.drop(['latitude'],axis=1)
X_train_lr= sm.add_constant(X_train_lr)
lr_3=sm.OLS(y_train, X_train_lr).fit()
print (lr_3.summary())

In [ ]:
vif = pd.DataFrame()
x = X_train_lr.drop(['const'],axis=1)
vif['Features'] =x.columns
vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range (x.shape[1])]
vif['VIF'] = round(vif['VIF'],2)
vif=vif.sort_values(by = 'VIF',ascending = False)
vif

***The above table indicates that VIF of minimum nights cloumn is high. Lets drop this column and run the model***

In [ ]:
X_train_re = X_train_lr.drop(['calculated_host_listings_count'],axis=1)
X_train_re= sm.add_constant(X_train_re)
lr_4=sm.OLS(y_train, X_train_re).fit()
print (lr_4.summary())

In [ ]:
vif = pd.DataFrame()
x = X_train_re.drop(['const'],axis=1)
vif['Features'] =x.columns
vif['VIF'] = [variance_inflation_factor(x.values,i) for i in range (x.shape[1])]
vif['VIF'] = round(vif['VIF'],2)
vif=vif.sort_values(by = 'VIF',ascending = False)
vif

Thus, this model is the final model where all variables are significant at 5% level with 53% R square. The most important variables explaining the variation in cnt variable are :



1. Shared Room
2. Private room
3. Manhathan
4. Brooklyn
5. availability_365
6. reviews_per_month
7. minimum_nights
.
Further the VIF value of all variables are also in within range (less than 6%)
Thus, this is the FINAL MODEL

### Step 6 : Residual Analysis of the training data 

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
y_train_pred = lr_4.predict(X_train_re)

In [ ]:
 #Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  
plt.xlabel('Errors', fontsize = 18)   

### Step 7 : Making Prediction on the testing data

In [ ]:
#df_test[Numeric] = scaler.transform(df_test[Numeric])
X_test = df_test.drop(['price'],1)
y_test = df_test['price']
df_test[num_vars] = scaler.transform(df_test[num_vars])

In [ ]:
X_test_new = X_test[['minimum_nights', 'reviews_per_month',
        'availability_365', 'feb', 'mar',
       'oct', 'Private room', 'Shared room', 'Brooklyn', 'Manhattan', 
       'Staten Island']]
X_test_new = sm.add_constant(X_test_new)
y_test_pred = lr_4.predict(X_test_new)



### Step 8 Model Evaluation 

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_test_pred)
fig.suptitle('y_test vs y_test_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_test_pred', fontsize = 16)     

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
r2_score( y_test ,y_test_pred)

In [ ]:
mse = mean_squared_error(y_test, y_test_pred)
mse
print('Test RMSE:',np.sqrt(mse))

## TESTING DATA 

In [ ]:
# Importing the testing file
df_1 = pd.read_csv('/content/drive/My Drive/Team Rocket/Test.csv')
# Looking at the first five rows
df_1.head()

In [ ]:
df_1.shape

In [ ]:
df_1.info()

In [ ]:
# columnwise null count for application data
df_1.isnull().sum()

In [ ]:
# Treating the null values
df_1.last_review.value_counts()
df_1['last_review'].fillna('2019-06-23',inplace = True)

In [ ]:
df_1.reviews_per_month = df_1.reviews_per_month.fillna(0)

In [ ]:
# Lets check the percentage of null values
df_col = round((df_1.isnull().sum()/len(df.index))*100,2)
print("column wise percentage in rental data", df_col)

### DATA PREPARATION

In [ ]:
df_1['last_review'] = pd.to_datetime(df_1.last_review,format='%Y-%m-%d')
df_1['Months'] = df_1.last_review.dt.strftime('%m')
df_1.Months.value_counts()

In [ ]:
#Lets map the variable Months
df_1['Months'] = df_1['Months'].astype(int)
df_1['Months'] = df_1['Months'].map({1:'jan',2:'feb',3:'mar', 4:'apr',5:'may',6:'jun', 7:'jul',8:'aug',9:'sep',10 :'oct',11:'nov',12:'dec'})
df_1.Months.value_counts()

In [ ]:
month_dummy = pd.get_dummies(df_1['Months'],drop_first = True)
df_1 = pd.concat([df_1,month_dummy], axis =1)

room_dummy = pd.get_dummies(df_1['room_type'],drop_first = True)
df_1 = pd.concat([df_1,room_dummy], axis =1)

neighbourhood_group_dummy = pd.get_dummies(df_1['neighbourhood_group'],drop_first = True)
df_1 = pd.concat([df_1,neighbourhood_group_dummy], axis =1)

In [ ]:
# Drop month, room_type and neighbourhood_group as we have created the dummies for it

df_1 = df_1.drop(['Months','room_type','neighbourhood_group'], axis = 1)
df_1.head()

In [ ]:
df_1.drop(['id','name','host_name','neighbourhood','last_review'],1,inplace=True)

In [ ]:
#Numeric = ['latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']
Numeric = ['minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365']

df_1[Numeric] = np.log1p(df_1[Numeric])
df_1


In [ ]:
df_1.head()

In [ ]:
df_1.info()

### PREDICTING THE PRICE ON THE TESTING DATA 

In [ ]:

X_testing= df_1[['minimum_nights', 'reviews_per_month',
        'availability_365', 'feb', 'mar',
       'oct', 'Private room', 'Shared room', 'Brooklyn', 'Manhattan', 
       'Staten Island']]
X_testing = sm.add_constant(X_testing)
y_pred = lr_4.predict(X_testing)

In [ ]:
y_pred

### 2. Cross Validation with K fold

In [ ]:
import re

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
lm = LinearRegression()
scores = cross_val_score(lm,X_train,y_train,scoring = "r2" , cv=5)
scores

In [ ]:
scores = cross_val_score(lm,X_train,y_train,scoring = "neg_mean_squared_error" , cv=5)
scores

In [ ]:
from sklearn.metrics import SCORERS
print(SCORERS.keys())

#### Hypertuning using Grid Search Cross Validation 

In [ ]:
len(X_train.columns)

In [ ]:
# step-1: create a cross-validation scheme
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(1, 26))}]


# step-3: perform grid search
# 3.1 specify model
lm = LinearRegression()
lm.fit(X_train, y_train)
rfe = RFE(lm)             

# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator = rfe, 
                        param_grid = hyper_params, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_cv.fit(X_train, y_train)                  


In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
# plotting cv results
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
# final model
n_features_optimal = 16

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=n_features_optimal)             
rfe = rfe.fit(X_train, y_train)

# predict prices of X_test
y_pred = lm.predict(X_test)
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

In [ ]:
mse = sklearn.metrics.mean_squared_error(y_test, y_pred)
print('Test RMSE:',np.sqrt(mse))


### 3. Random Forest Regression  

In [ ]:
from  sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor 


In [ ]:
dt = DecisionTreeRegressor (random_state =42,max_depth=4,min_samples_leaf =10)

In [ ]:
np.random.seed(0)
df_train,df_test =train_test_split(df,train_size=0.7,random_state=100)

In [ ]:
df_test.head()

In [ ]:
scaler=MinMaxScaler()

In [ ]:
y_train= df_train.pop('price')
X_train = df_train

y_test= df_test.pop('price')
X_test= df_test

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydotplus,graphviz

dot_data = StringIO()
export_graphviz(dt,out_file=dot_data,filled=True,rounded=True,feature_names=X_train.columns)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
y_train_pred=dt.predict(X_train)

In [ ]:
r2_score(y_train,y_train_pred)

In [ ]:
mse = mean_squared_error(y_train,y_train_pred)
print('Test RMSE:',np.sqrt(mse))

In [ ]:
y_test_pred = dt.predict(X_test)

In [ ]:
r2_score(y_test,y_test_pred)

In [ ]:
mse = mean_squared_error(y_test,y_test_pred)
print('Test RMSE:',np.sqrt(mse))

### USING RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf=RandomForestRegressor (random_state = 42,n_jobs=-1,max_depth=5,min_samples_leaf=10)

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
sample_tree=rf.estimators_[10]

In [ ]:
dot_data = StringIO()
export_graphviz(sample_tree,out_file=dot_data,filled=True,rounded=True,feature_names=X_train.columns)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
y_train_pred=rf.predict(X_train)
y_test_pred=rf.predict(X_test)
r2_score(y_train,y_train_pred)

In [ ]:
mse = mean_squared_error(y_train,y_train_pred)
print('Test RMSE:',np.sqrt(mse))

In [ ]:
r2_score(y_test,y_test_pred)

In [ ]:
mse = mean_squared_error(y_test,y_test_pred)
print('Test RMSE:',np.sqrt(mse))

In [ ]:
rf.feature_importances_

In [ ]:
important_features = pd.DataFrame({"Varnam" :X_train.columns,"Imp":rf.feature_importances_})

In [ ]:
important_features.sort_values(by='Imp',ascending =False).head(7)

### USING XGBoost

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(random_state=42)

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
y_pred_xgb = xgb.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test,y_pred_xgb)
print('Test RMSE:',np.sqrt(mse))

In [ ]:
# A parameter grid for XGBoost
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
from sklearn.model_selection import cross_val_score,GridSearchCV,RandomizedSearchCV

In [ ]:
xgb = XGBRegressor()

grid_xgb = RandomizedSearchCV(xgb, param_grid, n_iter=20,
                            n_jobs=-1, verbose=2, cv=3,                            
                             random_state=42)

In [ ]:
grid_xgb.fit(X_train,
         y_train)

In [ ]:
grid_xgb.best_estimator_

In [ ]:
xgb_ht = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1.0, verbosity=1)
xgb_ht.fit(X_train,y_train)
y_pred_xgb_ht = xgb_ht.predict(X_test)
#scores(y_test,y_pred_xgb_ht,'HyperTuned XGBoost')

In [ ]:
mse = mean_squared_error(y_test,y_pred_xgb_ht)
print('Test RMSE:',np.sqrt(mse))

In [ ]:
y_pred = xgb_ht.predict(df_1) 

In [ ]:
df_1.head(1)

In [ ]:
sample_submission = pd.DataFrame(y_pred,df_1.host_id,columns=['price'])
sample_submission = np.expm1(sample_submission)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv("submission_1.csv")
files.download('submission_1.csv')

In [ ]:
from google.colab import files

sample_submission.to_csv("submission_1.csv")
files.download('submission_1.csv')

After performing on various algorithm such as  vanilla model and hyperparameter tuning using Grid search and Random Forest,we found that Linear regression  is generating lesser RMSLE value as compare to other models.
Model   
1. Vanilla Model : RMSE is 0.072 
2. K Fold with cross validation :RMSE is 0.0687
3. Simple Decision Tree: RMSE is 0.379
4. Random Forest : RMSE is 0.379
5. XG Boost : RMSE is 0.356

Accordingly we have predicted the price on the GIVEN TEST DATA by running vanilla model. The important features and their interpretation:

1. Shared Room  : The corefficent is negative which indicates that with the in reference to apartment , the rental demand of shared room declines with the increase in prices. 


2. Private Room : Similarly, when compared with the apartment , the rental demand of private room is less.


3. Manhathan : The rental demand of Manhathan is more when compared to the Bronx.

4. minimum_night: With the incease in  minimum_nights, the price would declines

5. feb : In compared to other months, the rental demand declines in the month of feb

INTERPRETATION :
The above analysis indicates that in order to optimize prices, more investment should be incurred in apartments and Manhathon region. 

